In [ ]:
import sys
import json
from pathlib import Path

from tqdm import tqdm
import numpy as np
import pandas as pd

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Read subject data from local file

In [ ]:
df = pd.read_csv(ROOT / 'datasets' / 'drawing_evaluations_grouped.csv', index_col=0)

# Convert non-primitive fields and normalize paths
for col in df.columns:
    if col.startswith('processed_path_'):
        print(df[col])
        df[col] = df[col].apply(lambda x: ROOT / x)

df.head()

### Load custom MLP model

In [ ]:
from quick_draw_learner import QDClassifier

### Create a file to hold intermediate results

In [ ]:
results_file = ROOT / 'results' / 'qd_grouped.csv'

if not results_file.exists():
    pd.DataFrame(columns=[
        'Key',
        'Trial',
        'Subset splits', 
        'Kernel size', 
        'Batch size', 
        'Random seed', 
        'Accuracy',
        'Precision',
        'Recall',
        'Area under ROC']).set_index('Key').to_csv(results_file)

trial_results = pd.read_csv(results_file, index_col='Key')

### Build and train...

In [ ]:
from itertools import product, combinations

target_label = 'SANO'
target_column = 'diagnosis'
category_columns = [col for col in df.columns if col != target_column]

# Define all possible hyperparameters
# kernel_sizes = [3, 5, 7, 9, 11]
# batch_sizes = [24, 32, 48, 56, 64]
# test_splits = [.2, .25]
# validation_splits = [.2, .25]
kernel_sizes = [9]
batch_sizes = [48]
test_splits = [.2]
validation_splits = [.2]

# Initialize random number generator without seed to randomize hyperparamters
rnd = np.random.RandomState(seed=0)

# Cross product all hyperparameters
parameter_combinations = list(product(
    kernel_sizes, batch_sizes, test_splits, validation_splits))
rnd.shuffle(parameter_combinations)

category_columns

In [ ]:
from bananas.dataset import DataSet, DataType, Feature
from bananas.sampling.cross_validation import DataSplit
from bananas.statistics import scoring
from bananas.statistics.scoring import ScoringFunction

# Perform 3 trials per parameter set to later compute the average
trial_count = 3
score_history = {}

parameter_iterator = tqdm(parameter_combinations, leave=False)
for kernel_size, batch_size, test_split, validation_split in parameter_iterator:
    
    trial_params = {
        'Subset splits': (test_split, validation_split),
        'Kernel size': kernel_size,
        'Batch size': batch_size,
    }
    
    # Execute the independent trials
    for trial_num in tqdm(range(trial_count), leave=False, desc='Trial'):
        trial_params['Trial'] = trial_num
        
        # If these parameters have already been tried, we can skip trial
        trial_key = '|'.join(
            ['%s=%s' % (k, str(v)) for k, v in trial_params.items()])
        if trial_key in trial_results.index: continue
        score_history[trial_key] = []

        # Create a single feature containing all image data
        image_loader = ImageAugmenterMultiLoader(
            df[category_columns].values, cache=batch_size*32)
        features = [Feature(
            image_loader,
            kind=DataType.HIGH_DIMENSIOAL,
            sample_size=4,
            random_seed=0)]

        # Define target feature
        target_feature = Feature(
            (df[target_column] == target_label).values, random_seed=0)
        
        # Compute the overall class balance
        true_class_balance = sum(target_feature[:] / len(target_feature))


        # Always re-initialize the randopm seed
        random_seed = trial_num

        while True:

            # Change seed at every iteration to ensure different dataset split
            random_seed = np.random.RandomState(seed=random_seed).randint(1E6)

            # Build dataset, making sure that we have a left-out subset
            dataset = DataSet(
                features,
                name=target_label,
                target=target_feature,
                random_seed=random_seed,
                batch_size=batch_size,
                test_split=test_split,
                validation_split=validation_split)

            # Rebuild dataset unless test class balance is within 5% of ground truth
            test_idx = dataset.sampler.subsamplers[DataSplit.VALIDATION].data
            test_classes = target_feature[test_idx]
            test_class_balance = sum(test_classes) / len(test_classes)
            if abs(test_class_balance - true_class_balance) < .05: break

        # Instantiate learner using pre-trained model
        learner = QDClassifier(
            kernel_size=kernel_size,
            input_channel_count=len(category_columns),
            random_seed=0,
            verbose=False)

        # Train learner using train dataset
        learner.train(
            dataset.input_fn,
            progress=True,
            max_steps=500,
            callback=lambda x: score_history[trial_key].append(x.score)
        )

        # Test learner predictions using left-out validation dataset
        X, y = dataset[test_idx]
        y = learner.label_encoder_.transform(y)
        y_ = learner.predict_proba(X)

        # Store trial results
        trial_results.loc[trial_key] = {
            **trial_params,
            'Random seed': random_seed,
            'Accuracy': scoring.score_accuracy(y, y_),
            'Precision': scoring.score_precision(y, y_),
            'Recall': scoring.score_recall(y, y_),
            'Area under ROC': scoring.score_auroc(y, y_),
        }
        
        trial_results = trial_results.sort_values('Accuracy', ascending=False)
        trial_results.to_csv(results_file)

In [ ]:
trial_results.head(15)